In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold

In [2]:
%matplotlib inline

# Import data

In [3]:
# load data
rawTrain = pd.read_csv("../input/train.csv").astype(float)
rawTest = pd.read_csv("../input/test.csv").astype(float)

# extract labels and features 
Y = rawTrain.ix[:,0].values
rawX = rawTrain.ix[:,1:].values

# Prepare data

In [4]:
normX = normalize(rawX)
normTest = normalize(rawTest)
normX.shape

(42000L, 784L)

# Remove features with zero variance

In [5]:
selector = VarianceThreshold()
X = selector.fit_transform(normX)
testX = selector.transform(normTest)
X.shape

(42000L, 708L)

# Training

*** No "magic" numbers here, see Tuning_RF_KN for details ***

In [7]:
rf = RandomForestClassifier(n_estimators=100, bootstrap=False, min_samples_leaf=1, 
                            min_samples_split=3, criterion='gini', max_features=30)
kn = KNeighborsClassifier(n_neighbors=4, weights='distance')
voter = VotingClassifier(estimators=[('rf', rf), ('kn', kn)], voting='soft')

In [11]:
scores = cross_validation.cross_val_score(voter, X, Y, cv=3, n_jobs=-1)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.9737 (+/- 0.0021)


In [8]:
voter.fit(X, Y)

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=30, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob...wski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='distance'))],
         voting='soft', weights=None)

# Export

In [13]:
prediction = voter.predict(testX).astype(int)
result = pd.DataFrame(prediction, columns = ['Label'])
result.index += 1 
result.to_csv("../output/rf_kn.csv", index_label = "ImageId")